## Downloads required

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

%cd /content/drive/My Drive/Thesis/Thesis-submitting

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/My Drive/Thesis/Thesis-submitting'
/content/ta-lib/ta-lib


# Set up 

### Loading Libraries

In [ ]:
#Basics
import pandas as pd
from pandas import DataFrame
import numpy as np
import talib as ta
from datetime import datetime, timedelta

#Visuals
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,7)
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import seaborn as sns

#ARIMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

#Sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn import cross_decomposition
from sklearn.model_selection import cross_val_score, train_test_split


from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import BayesianRidge, ElasticNetCV, LinearRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor


from sklearn.metrics import mean_absolute_error, accuracy_score

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, SelectKBest, chi2, VarianceThreshold

import xgboost as xgb


# Default figure size
sns.set(rc={"figure.figsize": (12, 6)})

# Hoping to get rid of these
from scipy import stats
import statsmodels.api as sm
from itertools import product

import warnings
warnings.filterwarnings('ignore')

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

rand_seed = 1234

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

%cd /content/drive/My Drive/Thesis/Thesissub/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Thesis/Thesissub


## Load Data

In [ ]:
import pandas_datareader as pdr
import datetime
import math

In [ ]:
btc= pdr.DataReader('BTC-USD', 'yahoo')
eth= pdr.DataReader('ETH-USD', 'yahoo')

In [ ]:
btc = btc[:-1]
eth = eth[:-1]

In [ ]:
btc = btc.reset_index()
eth = eth.reset_index()

In [ ]:
for item in (btc, eth):
    item.columns = map(str.lower, item.columns)
    item['date'] = pd.to_datetime(item.date)
    #item.set_index('date', inplace=True)
    item = item.fillna(method="ffill")



# Feature Engineering



*   Lagged vars
*   Rolling means
*   Rolling variances







In [ ]:
#tf = tf.reset_index()
for df in (btc, eth):
  
  print("Creating a few lagged variables..")

  df['lag_1'] =  df['close'].shift()
  df['lag_2'] =  df['close'].shift(2)
  df['lag_3'] =  df['close'].shift(3)
  df['lag_4'] =  df['close'].shift(4)
  df['lag_5'] =  df['close'].shift(5)

  print("Creating overlapping items..")
  #add technical indicators - overlapping items
  # Bollinger bands
  df['upper_band'], df['middle_band'], df['lower_band'] = ta.BBANDS(df['close'], timeperiod =30)

  # To ensure the model focuses on the most current events rather than previous trends too heavily
  df['ewm_30'] = df["close"].ewm(span=30).mean()
  df['ewm_15'] = df["close"].ewm(span=15).mean()

  #df['HT_Trendline'] = ta.HT_TRENDLINE(df['close'])

  df['dema_15'] = ta.DEMA(df['close'],15)
  df['dema_30'] = ta.DEMA(df['close'],30)

  df['kama_15'] = ta.KAMA(df['close'],15)
  df['kama_30'] = ta.KAMA(df['close'],30)

  df['MIDPOINT_15'] = ta.MIDPOINT(df['close'],15)
  df['MIDPOINT_30'] = ta.MIDPOINT(df['close'],30)
  

  df['MIDPOINT_15'] = ta.MIDPOINT(df['close'],15)
  df['MIDPOINT_30'] = ta.MIDPOINT(df['close'],30)

  df['SAR'] =ta.SAR(df['high'], df['low'], 0, 0.2)

  df['SMA_5'] = df['close'].rolling(5).mean()
  df['SMA_7'] = df['close'].rolling(7).mean()
  df['SMA_14'] = df['close'].rolling(14).mean()
  df['SMA_30'] = df['close'].rolling(30).mean()

  
  #df['tema_15'] = ta.TEMA(df['close'],15)
  #df['tema_30'] = ta.TEMA(df['close'],30)

  df['trima_15'] = ta.TRIMA(df['close'],15)
  df['trima_30'] = ta.TRIMA(df['close'],30)

  df['WMA_5'] = ta.WMA(df['close'],5)
  df['WMA_7'] = ta.WMA(df['close'],7)
  df['WMA_14'] = ta.WMA(df['close'],14)
  df['WMA_30'] = ta.WMA(df['close'],30)

  print("Variances..")

  df['roll_var_5'] = df['close'].rolling(5).var()
  df['roll_var_7'] = df['close'].rolling(7).var()
  df['roll_var_14'] = df['close'].rolling(14).var()
  df['roll_var_30'] = df['close'].rolling(30).var()
 
  print("Standard deviations..")
  # set .std(ddof=0) for population std instead of sample
  df['5 Day STD'] = df['close'].rolling(window=5).std()
  df['7 Day STD'] = df['close'].rolling(window=7).std()
  df['14 Day STD'] = df['close'].rolling(window=14).std() 
  df['30 Day STD'] = df['close'].rolling(window=30).std() 



# Plot
  print("Done!")


Creating a few lagged variables..
Creating overlapping items..
Variances..
Standard deviations..
Done!
Creating a few lagged variables..
Creating overlapping items..
Variances..
Standard deviations..
Done!


## Momentum indicators

In [ ]:
# Momentum indicators
for df in (btc, eth):

  df['ADX_5'] = ta.ADX(df['high'], df['low'], df['close'],5)
  df['ADX_10'] = ta.ADX(df['high'], df['low'], df['close'],10)
  df['ADX_30'] = ta.ADX(df['high'], df['low'], df['close'],30)

  df['ADXR_10'] = ta.ADXR(df['high'], df['low'], df['close'],10)

  df['APO'] = ta.APO(df.close, fastperiod=10, slowperiod=30, matype = 0)

  #df['Aroonup_10'] = ta.AROONOSC(df['close'],10)
  df['aroondown'], df['aroonup'] = ta.AROON(df['high'], df['low'], timeperiod=10)
  df['aroon_osc'] = ta.AROONOSC(df['high'], df['low'], timeperiod=10)

  df['BOP'] = ta.BOP(df['open'],df['high'], df['low'], df['close'])

  df['CCI_30'] = ta.CCI(df['high'], df['low'], df['close'],30)

  df['CMO_10'] = ta.CMO(df['close'],10)

  df['DX_10'] = ta.DX(df['high'], df['low'], df['close'],10)

  df['macd'], df['macdsignal'], df['macdhist'] = ta.MACD(df['close'], fastperiod=10, slowperiod=30, signalperiod=5)

  #df['MACD_5'] = ta.MACD(df['close'],5)
  #df['MACD_10'] = ta.MACD(df['close'],10)
  #df['MACD_30'] = ta.MACD(df['close'],30)

  df['MINUS_DI_10'] = ta.MINUS_DI(df['high'], df['low'], df['close'],10)
  df['MINUS_DM_10'] = ta.MINUS_DM(df['high'], df['low'],10)

  df['MOM_10'] = ta.MOM(df['close'],10)


  df['PLUS_DI_10'] = ta.PLUS_DI(df['high'], df['low'], df['close'],10)
  df['PLUS_DM_10'] = ta.PLUS_DM(df['high'], df['low'],10)

  df['PPO'] = ta.PPO(df['close'], fastperiod=10, slowperiod=30, matype=0)


  df['roc_5'] = ta.ROC(df['close'], 5)
  df['roc_10'] = ta.ROC(df['close'], 10)
  df['roc_30'] = ta.ROC(df['close'], 30)

  df['rsi_5'] = ta.RSI(df['close'], 5)
  df['rsi_10'] = ta.RSI(df['close'], 10)
  df['rsi_30'] = ta.RSI(df['close'], 30)

  #add stochastc oscillators 
  df['slowk'], df['slowd'] = ta.STOCH(df['high'], df['low'], df['close'], fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
  df['fastk'], df['fastd'] = ta.STOCHF(df['high'], df['low'], df['close'], fastk_period=14, fastd_period=3, fastd_matype=0)

  df['fastk_rsi'], df['fastd_rsi']  = ta.STOCHRSI(df['close'], timeperiod=10, fastk_period=5, fastd_period=3, fastd_matype=0)

  df['TRIX_10'] = ta.TRIX(df['close'], 10)

  df['ULTOSC'] = ta.ULTOSC(df['high'], df['low'], df['close'], timeperiod1=5, timeperiod2=10, timeperiod3=30)
 
  df['WILLR'] = ta.WILLR(df['high'], df['low'], df['close'],10)
  
  print("Oscillators added..")

  df['HT_DCPERIOD'] = ta.HT_DCPERIOD(df['close'])
  #df['HT_DCPHASE'] = ta.HT_DCPHASE(df['close'])
  df['HT_DCTREND'] = ta.HT_TRENDMODE(df['close'])

  print("Cycle indicators added..")


Oscillators added..
Cycle indicators added..
Oscillators added..
Cycle indicators added..


In [ ]:
for df in (btc, eth):
  df = df.drop(['high', 'low', 'open', 'volume', 'adj close'], axis = 1)

# Save dataframe

In [ ]:
btc = btc.drop(['high', 'low', 'open', 'volume', 'adj close'], axis = 1)
eth = eth.drop(['high', 'low', 'open', 'volume', 'adj close'], axis = 1)

btc = btc.dropna()
eth = eth.dropna()

## Datetime conversion
btc['date'] = pd.to_datetime(btc.date)
## Datetime conversion
eth['date'] = pd.to_datetime(eth.date)

In [ ]:
cd Thesis\ -\ submitting/

[Errno 2] No such file or directory: 'Thesis - submitting/'
/content/ta-lib/ta-lib


In [ ]:
btc.to_csv('btc_w_features.csv')

eth.to_csv('eth_w_features.csv')

# Feature selection

In [ ]:
#!pip install eli5

In [ ]:
#eth = eth.drop(['index'], axis =1)

In [ ]:
eth = eth.set_index('date')

In [ ]:
eth.head()

,close,lag_1,lag_2,lag_3,lag_4,lag_5,upper_band,middle_band,lower_band,ewm_30,ewm_15,dema_15,dema_30,kama_15,kama_30,MIDPOINT_15,MIDPOINT_30,SAR,SMA_5,NEW_SMA_5,SMA_7,SMA_14,SMA_30,trima_15,trima_30,WMA_5,WMA_7,WMA_14,WMA_30,roll_var_5,roll_var_7,roll_var_14,roll_var_30,5 Day STD,7 Day STD,14 Day STD,30 Day STD,ADX_5,ADX_10,ADX_30,ADXR_10,APO,aroondown,aroonup,aroon_osc,BOP,CCI_30,CMO_10,DX_10,macd,macdsignal,macdhist,MINUS_DI_10,MINUS_DM_10,MOM_10,PLUS_DI_10,PLUS_DM_10,PPO,roc_5,roc_10,roc_30,rsi_5,rsi_10,rsi_30,slowk,slowd,fastk,fastd,fastk_rsi,fastd_rsi,TRIX_10,ULTOSC,WILLR,HT_DCPERIOD,HT_DCTREND
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-10-27,0.869641,0.731317,0.616039,0.563590,0.539657,0.567702,0.787714,0.610330,0.432947,0.644997,0.619304,0.629826,0.528160,0.615504,0.587881,0.652235,0.652235,1.39615,0.664049,0.664049,0.619325,0.567526,0.610330,0.529684,0.591955,0.719228,0.678987,0.603524,0.596478,0.018666,0.019485,0.012718,0.008137,0.136623,0.139590,0.112776,0.090208,58.970293,36.746349,31.069537,38.297876,-0.032645,40.0,100.0,60.0,0.849267,227.502281,49.054821,55.241774,-0.028652,-0.075500,0.046848,11.788948,0.114061,0.322463,40.889406,0.395616,-5.348770,53.186181,58.932021,20.642887,88.631801,74.527411,51.978621,86.415021,74.230857,94.022102,86.415021,100.00000,100.000000,-0.458187,62.485553,-5.977898,24.089428,0
2015-10-28,1.002480,0.869641,0.731317,0.616039,0.563590,0.539657,0.850339,0.624317,0.398295,0.668462,0.667215,0.718284,0.579158,0.657627,0.614422,0.718655,0.718655,1.39615,0.756613,0.756613,0.698632,0.601777,0.624317,0.547184,0.588584,0.832039,0.774776,0.661518,0.621778,0.032721,0.031685,0.025855,0.013212,0.180890,0.178003,0.160795,0.114943,64.999495,39.740526,30.599149,40.724075,0.001843,30.0,100.0,70.0,0.530555,308.521368,59.633347,66.688116,0.016200,-0.044933,0.061133,9.176230,0.102655,0.484746,45.916599,0.513673,0.295197,85.762445,93.628397,71.985614,92.179157,79.816674,56.112958,92.614056,84.727295,91.603322,92.614056,100.00000,100.000000,0.329190,69.541358,-8.396678,24.656104,0
2015-10-29,1.206660,1.002480,0.869641,0.731317,0.616039,0.563590,0.950395,0.642501,0.334606,0.703749,0.734663,0.838305,0.651046,0.778918,0.663727,0.820745,0.820745,1.39615,0.885227,0.885227,0.789912,0.647833,0.642501,0.579054,0.587958,0.982054,0.901783,0.742169,0.659349,0.053365,0.062123,0.051593,0.024517,0.231009,0.249244,0.227141,0.156579,70.484200,43.147588,30.428947,43.639467,0.055424,20.0,100.0,80.0,0.776787,362.299154,70.200825,73.811153,0.074291,-0.005192,0.079483,7.297054,0.092390,0.717646,48.429368,0.613175,8.626259,114.102460,146.753678,82.510377,95.110536,85.100412,61.397394,95.208474,91.412517,100.000000,95.208474,100.00000,100.000000,1.356837,75.799520,-0.000000,25.887397,0
2015-10-30,1.041220,1.206660,1.002480,0.869641,0.731317,0.616039,0.990753,0.652586,0.314420,0.725852,0.772991,0.897177,0.695612,0.805329,0.676195,0.820745,0.820745,1.39615,0.970264,0.970264,0.861564,0.683885,0.652586,0.617623,0.589906,1.034052,0.964610,0.794621,0.685072,0.032294,0.056221,0.061144,0.029575,0.179706,0.237109,0.247273,0.171974,75.263481,46.682561,30.485116,46.569007,0.105977,10.0,100.0,90.0,-0.445701,291.899471,37.737935,78.497318,0.098475,0.029364,0.069111,5.483426,0.083151,0.606391,45.518825,0.690248,16.239533,69.018518,139.455040,40.963704,68.939363,68.868967,55.768747,86.242242,91.354924,67.123405,86.242242,13.96221,71.320737,2.221777,63.811425,-32.876595,26.350540,0
2015-10-31,0.916627,1.041220,1.206660,1.002480,0.869641,0.731317,1.011183,0.660134,0.309084,0.738335,0.790949,0.915300,0.720493,0.812227,0.681471,0.820745,0.820745,1.39615,1.007326,1.007326,0.911998,0.710274,0.660134,0.662430,0.594174,1.016173,0.978376,0.825653,0.702107,0.017022,0.038961,0.063123,0.031871,0.130470,0.197385,0.251243,0.178525,73.763424,48.275674,30.349120,45.301852,0.145360,0.0,90.0,90.0,-0.831917,167.784720,18.780449,62.613685,0.101216,0.053315,0.047901,9.390366,0.142825,0.469298,40.843878,0.621223,22.019745,25

In [ ]:
TA= eth

In [ ]:
TA.head()

,close,lag_1,lag_2,lag_3,lag_4,lag_5,upper_band,middle_band,lower_band,ewm_30,ewm_15,dema_15,dema_30,kama_15,kama_30,MIDPOINT_15,MIDPOINT_30,SAR,SMA_5,NEW_SMA_5,SMA_7,SMA_14,SMA_30,trima_15,trima_30,WMA_5,WMA_7,WMA_14,WMA_30,roll_var_5,roll_var_7,roll_var_14,roll_var_30,5 Day STD,7 Day STD,14 Day STD,30 Day STD,ADX_5,ADX_10,ADX_30,ADXR_10,APO,aroondown,aroonup,aroon_osc,BOP,CCI_30,CMO_10,DX_10,macd,macdsignal,macdhist,MINUS_DI_10,MINUS_DM_10,MOM_10,PLUS_DI_10,PLUS_DM_10,PPO,roc_5,roc_10,roc_30,rsi_5,rsi_10,rsi_30,slowk,slowd,fastk,fastd,fastk_rsi,fastd_rsi,TRIX_10,ULTOSC,WILLR,HT_DCPERIOD,HT_DCTREND
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-10-27,0.869641,0.731317,0.616039,0.563590,0.539657,0.567702,0.787714,0.610330,0.432947,0.644997,0.619304,0.629826,0.528160,0.615504,0.587881,0.652235,0.652235,1.39615,0.664049,0.664049,0.619325,0.567526,0.610330,0.529684,0.591955,0.719228,0.678987,0.603524,0.596478,0.018666,0.019485,0.012718,0.008137,0.136623,0.139590,0.112776,0.090208,58.970293,36.746349,31.069537,38.297876,-0.032645,40.0,100.0,60.0,0.849267,227.502281,49.054821,55.241774,-0.028652,-0.075500,0.046848,11.788948,0.114061,0.322463,40.889406,0.395616,-5.348770,53.186181,58.932021,20.642887,88.631801,74.527411,51.978621,86.415021,74.230857,94.022102,86.415021,100.00000,100.000000,-0.458187,62.485553,-5.977898,24.089428,0
2015-10-28,1.002480,0.869641,0.731317,0.616039,0.563590,0.539657,0.850339,0.624317,0.398295,0.668462,0.667215,0.718284,0.579158,0.657627,0.614422,0.718655,0.718655,1.39615,0.756613,0.756613,0.698632,0.601777,0.624317,0.547184,0.588584,0.832039,0.774776,0.661518,0.621778,0.032721,0.031685,0.025855,0.013212,0.180890,0.178003,0.160795,0.114943,64.999495,39.740526,30.599149,40.724075,0.001843,30.0,100.0,70.0,0.530555,308.521368,59.633347,66.688116,0.016200,-0.044933,0.061133,9.176230,0.102655,0.484746,45.916599,0.513673,0.295197,85.762445,93.628397,71.985614,92.179157,79.816674,56.112958,92.614056,84.727295,91.603322,92.614056,100.00000,100.000000,0.329190,69.541358,-8.396678,24.656104,0
2015-10-29,1.206660,1.002480,0.869641,0.731317,0.616039,0.563590,0.950395,0.642501,0.334606,0.703749,0.734663,0.838305,0.651046,0.778918,0.663727,0.820745,0.820745,1.39615,0.885227,0.885227,0.789912,0.647833,0.642501,0.579054,0.587958,0.982054,0.901783,0.742169,0.659349,0.053365,0.062123,0.051593,0.024517,0.231009,0.249244,0.227141,0.156579,70.484200,43.147588,30.428947,43.639467,0.055424,20.0,100.0,80.0,0.776787,362.299154,70.200825,73.811153,0.074291,-0.005192,0.079483,7.297054,0.092390,0.717646,48.429368,0.613175,8.626259,114.102460,146.753678,82.510377,95.110536,85.100412,61.397394,95.208474,91.412517,100.000000,95.208474,100.00000,100.000000,1.356837,75.799520,-0.000000,25.887397,0
2015-10-30,1.041220,1.206660,1.002480,0.869641,0.731317,0.616039,0.990753,0.652586,0.314420,0.725852,0.772991,0.897177,0.695612,0.805329,0.676195,0.820745,0.820745,1.39615,0.970264,0.970264,0.861564,0.683885,0.652586,0.617623,0.589906,1.034052,0.964610,0.794621,0.685072,0.032294,0.056221,0.061144,0.029575,0.179706,0.237109,0.247273,0.171974,75.263481,46.682561,30.485116,46.569007,0.105977,10.0,100.0,90.0,-0.445701,291.899471,37.737935,78.497318,0.098475,0.029364,0.069111,5.483426,0.083151,0.606391,45.518825,0.690248,16.239533,69.018518,139.455040,40.963704,68.939363,68.868967,55.768747,86.242242,91.354924,67.123405,86.242242,13.96221,71.320737,2.221777,63.811425,-32.876595,26.350540,0
2015-10-31,0.916627,1.041220,1.206660,1.002480,0.869641,0.731317,1.011183,0.660134,0.309084,0.738335,0.790949,0.915300,0.720493,0.812227,0.681471,0.820745,0.820745,1.39615,1.007326,1.007326,0.911998,0.710274,0.660134,0.662430,0.594174,1.016173,0.978376,0.825653,0.702107,0.017022,0.038961,0.063123,0.031871,0.130470,0.197385,0.251243,0.178525,73.763424,48.275674,30.349120,45.301852,0.145360,0.0,90.0,90.0,-0.831917,167.784720,18.780449,62.613685,0.101216,0.053315,0.047901,9.390366,0.142825,0.469298,40.843878,0.621223,22.019745,25

In [ ]:
X = TA.iloc[:,0:(len(TA.columns)-1)]
y = TA.iloc[:,-1]

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.8, random_state = 42, shuffle= False)

## Use Random forrest regressor to figure out the best feature

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42)

In [ ]:
rf.fit(X_train, y_train)

print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf.score(X_train, y_train), 
                                                                                             rf.oob_score_,
                                                                                             rf.score(X_valid, y_valid)))

R^2 Training Score: 0.91 
OOB Score: 0.33 
R^2 Validation Score: -2.71


# Use elif to get the best and worst feature

In [ ]:
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rf, cv = None, refit = False, n_iter = 50).fit(X_train, y_train)
perm_imp_eli5 = imp_df(X_train.columns, perm.feature_importances_)

In [ ]:
from sklearn.base import clone 

def drop_col_feat_imp(model, X_train, y_train, random_state = 42):
    
    # clone the model to have the exact same specification as the one initially trained
    model_clone = clone(model)
    # set random_state for comparability
    model_clone.random_state = random_state
    # training and scoring the benchmark model
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)
    # list for storing feature importances
    importances = []
    
    # iterating over all columns and storing feature importance (difference between benchmark and new model)
    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis = 1), y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis = 1), y_train)
        importances.append(benchmark_score - drop_col_score)
    
    importances_df = imp_df(X_train.columns, importances)
    return importances_df

In [ ]:
drop_imp = drop_col_feat_imp(rf, X_train, y_train)
#var_imp_plot(drop_imp, 'Drop Column feature importance')

In [ ]:
drop_imp.sort_values(by= ['feature_importance'])

drop_imp.to_csv('drop_features_list_ETH.csv')

In [ ]:
drop_imp = drop_imp.set_index('feature')

In [ ]:
to_del = drop_imp.drop("close", axis =0)

In [ ]:
to_del = to_del[(to_del['feature_importance']<0)]

In [ ]:
to_del= to_del.reset_index()

In [ ]:
del_list = to_del["feature"].tolist()


In [ ]:
clean = TA.drop(del_list, axis=1)
clean = clean.reset_index()

In [ ]:
len(eth.columns)

75

In [ ]:
len(clean.columns)

62

In [ ]:
clean.tail(2)

,date,close,lag_3,lower_band,MIDPOINT_15,MIDPOINT_30,SAR,SMA_5,NEW_SMA_5,SMA_7,SMA_30,trima_15,WMA_5,WMA_7,WMA_14,WMA_30,roll_var_5,roll_var_7,roll_var_14,roll_var_30,5 Day STD,7 Day STD,14 Day STD,30 Day STD,ADX_5,ADX_10,ADX_30,ADXR_10,APO,aroondown,aroonup,aroon_osc,BOP,CCI_30,CMO_10,DX_10,macd,macdsignal,macdhist,MINUS_DI_10,MINUS_DM_10,MOM_10,PLUS_DI_10,PLUS_DM_10,PPO,roc_5,roc_10,roc_30,rsi_5,rsi_10,rsi_30,slowk,slowd,fastk,fastd,fastk_rsi,fastd_rsi,TRIX_10,ULTOSC,WILLR,HT_DCPERIOD,HT_DCTREND
1763,2020-08-24,384.001038,395.835144,314.565787,408.890945,373.520721,0.420897,393.698242,393.698242,398.770682,386.488641,415.798703,393.835472,395.852066,405.995695,401.136940,83.385851,138.928776,369.088330,1337.818171,9.131585,11.786805,19.211672,36.576197,35.111788,29.157194,33.791461,41.752611,21.349554,100.0,20.0,-80.0,-0.704386,9.338723,-9.183943,14.167714,23.709375,30.478093,-6.768719,26.671144,57.962209,-49.353882,20.051599,43.576495,5.523980,-7.789542,-11.388790,26.292550,35.368950,45.408029,58.119860,34.683439,35.357515,20.973978,34.683439,0.000000,23.076711,0.531583,47.749088,-85.893952,21.264486,1
1764,2020-08-25,386.466125,391.384491,323.024299,410.699432,377.027542,0.420897,393.166199,393.166199,395.913875,389.049392,412.985749,391.424766,392.775927,402.582601,401.135487,90.882325,144.776943,385.326467,1127.408490,9.533222,12.032329,19.629734,33.576904,35.266859,27.658246,33.212910,40.075451,14.056755,90.0,10.0,-80.0,0.189003,-5.867066,-6.908411,14.167714,20.460497,27.138895,-6.678398,24.988852,52.165988,-47.320496,18.786838,39.218846,3.613103,-0.683638,-10.908703,24.809977,38.032917,46.545795,58.468996,30.230152,34.914378,17.246280,30.230152,10.849386,24.333709,0.395038,55.491346,-82.288948,20.772657,1


In [ ]:
#clean = clean.drop(['SAR'], axis =1)

In [ ]:
ls

 btc_w_features.csv  'Data Prep.ipynb'
 cleaned_BTC.csv      drop_features_list_BTC.csv
 cleaned_ETH.csv      drop_features_list_ETH.csv
 cleaned_X_BTC.csv   'EDA_running on ETH.ipynb'
 cleaned_X_ETH.csv    eth_w_features.csv
 cleaned_y_BTC.csv   'Sklearn models.ipynb'
 cleaned_y_ETH.csv   'Windows of Data_models.ipynb'


In [ ]:
clean.to_csv('cleaned_ETH.csv')